In [12]:
# load approved_nrp_ids
with open('approved_nrp_ids.txt', 'r') as f:
    approved_nrp_ids = [nrp_id.strip() for nrp_id in f.readlines()]


Approved NRP IDs (127 in total):


In [13]:
# load matches table
import pandas as pd
matches_table = pd.read_csv('matches_inspection_table.tsv', sep='\t')
# q: correct table: for each value x in 'NRP variant' column x -> x.split('#')[0]
matches_table['NRP variant'] = matches_table['NRP variant'].apply(lambda x: x.split('#')[0])


In [14]:
# q: get all NRP variants that are not in approved_nrp_ids but have a Verdict in ['good', 'to be corrected', 'was corrected']

matches_table_good = matches_table[matches_table['Verdict'].isin(['good', 'to be corrected', 'was corrected'])]
nrp_variants_good = set(matches_table_good['NRP variant'].unique())
print(f'NRP variants with verdicts good, to be corrected, was corrected ({len(nrp_variants_good)} in total):')
nrp_variants_approved = set(approved_nrp_ids)
print(f'Approved NRP variants ({len(nrp_variants_approved)} in total):')
nrp_variants_to_be_approved = sorted(nrp_variants_good - nrp_variants_approved)
print(f'NRP variants to be approved ({len(nrp_variants_to_be_approved)} in total):')
print(nrp_variants_to_be_approved)

NRP variants with verdicts good, to be corrected, was corrected (238 in total):
['BGC0000289.0', 'BGC0000296.0', 'BGC0000305.0', 'BGC0000306.0', 'BGC0000307.0', 'BGC0000310.0', 'BGC0000311.0', 'BGC0000313.0', 'BGC0000314.0', 'BGC0000315.0', 'BGC0000315.1', 'BGC0000315.2', 'BGC0000315.3', 'BGC0000315.4', 'BGC0000315.6', 'BGC0000322.0', 'BGC0000323.0', 'BGC0000334.0', 'BGC0000336.0', 'BGC0000339.0', 'BGC0000342.0', 'BGC0000346.0', 'BGC0000349.0', 'BGC0000354.1', 'BGC0000354.3', 'BGC0000359.0', 'BGC0000359.1', 'BGC0000368.0', 'BGC0000374.4', 'BGC0000374.6', 'BGC0000379.0', 'BGC0000383.0', 'BGC0000385.0', 'BGC0000386.1', 'BGC0000386.3', 'BGC0000393.0', 'BGC0000395.0', 'BGC0000398.0', 'BGC0000399.0', 'BGC0000399.1', 'BGC0000400.0', 'BGC0000402.0', 'BGC0000402.1', 'BGC0000402.2', 'BGC0000405.0', 'BGC0000405.1', 'BGC0000405.3', 'BGC0000416.0', 'BGC0000416.1', 'BGC0000423.0', 'BGC0000424.1', 'BGC0000426.0', 'BGC0000429.0', 'BGC0000429.1', 'BGC0000433.0', 'BGC0000434.0', 'BGC0000434.1', 'BGC000

In [15]:
from pathlib import Path
to_be_approved_bgc_ids = sorted({nrp_id.split('.')[0] for nrp_id in nrp_variants_to_be_approved})
mibig_results_dir = '/home/ilianolhin/programs/antismash7/mibig_results/'
as_paths = [mibig_results_dir + bgc_id for bgc_id in to_be_approved_bgc_ids]
with open('to_be_approved_as_paths.txt', 'w') as f:
    f.write('\n'.join(as_paths))

In [16]:
import json
rban_records_dir = Path('/home/ilianolhin/git/nerpa2_old/test_data/rban_records/')
rban_records = []
for nrp_id in nrp_variants_to_be_approved:
    with open(rban_records_dir / f'{nrp_id}.json', 'r') as f:
        rban_records.append(json.load(f))

with open('to_be_approved_rban_records.json', 'w') as f:
    json.dump(rban_records, f, indent=4)
